<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description-Task-1:-RNN-Language-Modelling-(30-+10-Points)" data-toc-modified-id="Description-Task-1:-RNN-Language-Modelling-(30-+10-Points)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description Task 1: RNN Language Modelling (30 +10 Points)</a></span><ul class="toc-item"><li><span><a href="#1a)-Language-Modelling-(30-Points)" data-toc-modified-id="1a)-Language-Modelling-(30-Points)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1a) Language Modelling (30 Points)</a></span></li><li><span><a href="#Conditional-Generation-(10-Points)" data-toc-modified-id="Conditional-Generation-(10-Points)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Conditional Generation (10 Points)</a></span></li></ul></li><li><span><a href="#Code-for-Task-1" data-toc-modified-id="Code-for-Task-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code for Task 1</a></span><ul class="toc-item"><li><span><a href="#Task-1.1" data-toc-modified-id="Task-1.1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Task 1.1</a></span></li></ul></li></ul></div>

# Natural Language Understanding: Project 1

[__Natural Language Understanding, Spring 2018, ETHZ__](http://www.da.inf.ethz.ch/teaching/2018/NLU/)

[__Project 1__ (ETHZ network)](http://www.da.inf.ethz.ch/teaching/2018/NLU/material/project.pdf)

## Description Task 1: RNN Language Modelling (30 +10 Points)

### 1a) Language Modelling (30 Points)
Your task is to build a simple LSTM language model. To be precise, we assume that words are independent given the recurrent hidden state; we compute a new hidden state given the last hidden state and last word, and predict the next word given the hidden state:
$$ P(w_1,\dots,w_n) = 􏰀\Pi_{t=1}^{n}(w_t|\mathbf{h_t})$$
$$ P(w_t|h_t) = \text{softmax}(\mathbf{Wh}_t)$$
$$ \mathbf{h_t} = f(\mathbf{h_{t−1}}, w_{t-1}^{*})$$

where $f$ is the LSTM recurrent function, $\mathbf{W} \in \mathbb{R}^{|V|×d}$ are softmax weights and $\mathbf{h_0}$ is either an all-zero constant or a trainable parameter.
You can use the tensorflow cell implementation __[1](https://www.tensorflow.org/api_docs/python/tf/nn/rnn_cell)__ to carry out the recurrent computation in $f$. However, you must construct the actual RNN yourself (e.g. don’t use tensorflow’s `static_rnn` or `dynamic_rnn` or any other RNN library). That means, you will need to use a python loop that sets up the unrolled graph. To make your life simpler, please follow these design choices:

__Model and Data specification__

- Use a special sentence-beginning symbol `<bos>` and a sentence-end symbol `<eos>` (please use exactly these, including brackets). The `<bos>` symbol is the input, when predicting the first word and the `<eos>` symbol you require your model to predict at the end of every sentence.
- Use a maximum sentence length of 30 (including the `<bos>` and `<eos>` symbol). Ignore longer sentences during training and testing.
- Use a special padding symbol `<pad>` (please use exactly this, including brackets) to fill up sentences of length shorter than 30. This way, all your input will have the same size.
- Use a vocabulary consisting of the 20K most frequent words in the training set, including the symbols `<bos>`, `<eos>`, `<pad>` and `<unk>`. Replace out-of-vocabulary words with the `<unk>` symbol before feeding them into your network (don’t change the file content).
- Provide the ground truth last word as input to the RNN, not the last word you predicted. This is common practice.
- Language models are usually trained to minimize the cross-entropy. Use tensorflow’s `tf.nn.sparse_softmax_cross_entropy_with_logits` to compute the loss (*This operation fuses the computation of the soft-max and the cross entropy loss given the logits. For numerical stability, it’s very important to use this function.*). Use the AdamOptimizer with default parameters to minimize the loss. Use `tf.clip_by_global_norm` to clip the norm of the gradients to 5.
- Use a batch size of 64.
- Use the data at __[6](https://polybox.ethz.ch/index.php/s/qUc2NvUh2eONfEB)__. Don’t pre-process the input further. All the data is already white-space tokenized and
lower-cased. One sentence per line.
- To initialize your weight matrices, use the `tf.contrib.layers.xavier_initializer()` initializer introduced in __[5](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf)__.

__Experiments__
All experiments should not run for longer than, say, four hours on the GPU. For this task, your
grade won’t improve with performance.

- __Experiment A__: Train your model with word-embedding dimensionality of 100 and a hidden state size of 512 and compute sentence perplexity on the evaluation set (see submission format below).
- __Experiment B__: It is common practice, to pretrain word embeddings using e.g. `word2vec`. This should make your model train faster as words will come already with some useful representation. Use the code at __[3](http://da.inf.ethz.ch/teaching/2018/NLU/material/load_embeddings.py)__ to load these word embeddings __[4](https://polybox.ethz.ch/index.php/s/cpicEJeC2G4tq9U)__ trained on the same corpus. Train your model again and compute evaluation perplexity.
- __Experiment C__: It is often desirable to make the LSTM more powerful, by increasing the hidden dimensionality. However, this will naturally increase the parameters $\mathbf{W}$ of the softmax. As a compromise, one can use a larger hidden state, but down-project it before the softmax. Increase the hidden state dimensionality from 512 to 1024, but down-project $h_t$ to dimensionality 512 before predicting $w_t$ as in
$$ \mathbf{\tilde{h}}_t = \mathbf{W}_P\mathbf{h}_t$$
where $W_P$ are parameters. Train your model again and compute evaluation perplexity.

__Submission and grading__
- Grading scheme: 100% correctness.
- Deadline April 20th, 23:59:59.
- You are not allowed to copy-paste any larger code blocks from existing implementations.
- Hand in
    - Your python code
    - __Three__ result files containing sentence-level perplexity numbers on the __test__ set (to be distributed) for
all three experiments. Recall that perplexity of a sentence $S = ⟨w_1, \dots , w_n⟩$ with respect to your model $p(w_t|w_1, \dots, w_{t−1})$ is defined as
$$ \text{Perp} = 2^{-\frac{1}{n} \sum_{t=1}^{n}\log_2 p(w_t|w_1,\dots,w_{t−1})}$$
The `<eos>` symbol is part of the sequence, while the `<pad>` symbols (if any) are not. Be sure to have the basis of the exponential and the logarithm match.<br>
__Input format sentences.test__<br>
One sentence (none of them is longer than 28 tokens) per line:<br>
         ```beside her , jamie bounced on his seat .
         i looked and saw claire montgomery looking up at me .
         people might not know who alex was , but they knew to listen to him .```<br>
__Required output format groupXX.perplexityY__<br>
(where XX is your group __number__ and Y ∈ {A,B,C} is the experiment). One perplexity number per line:<br>
         $10.232$<br>
         $2.434$<br>
         $5.232$<br>
Make sure to have equally many lines in the output as there are in the input – otherwise your submission will be rejected automatically.
    - You have to submit at https://cmt3.research.microsoft.com/NLUETHZ2018

### Conditional Generation (10 Points)
Let’s use your trained language model from above to generate sentences. Given an initial sequence of words, your are asked to __greedily__ generate words until either your model decides to finish the sentence (it generated `<eos>`) or a given maximum length has been reached. Note, that this task does not involve any training. Please see the tensorflow documentation on how to save and restore your model from above.
There are several ways how to implement the generation. For example, you can define a graph that computes just one step of the RNN given the last input and the last state (both from a new placeholder).
$$ \text{state}_t, p_t = f(\text{state}_{t−1},w_{t−1}) $$
That means, for a prefix of size $m$ and a desired length of $n$, you run this graph $n$ times. The first $m + 1$ times you take the input form the prefix. For the rest of the sequence, you take the most likely2 word $w^{t−1} = \text{argmax}_w p_{t−1}(w)$ from the last step.

- Grading scheme: 100% correctness.
- Deadline April 20th, 23:59:59.
- You are not allowed to copy-paste any larger code blocks from existing implementations.
- Hand in
    - Your python code
    - Your continued sentences of length up to 20. Use your trained model from experiment __C__ in task 1.1.
    __Input format sentences.continuation__ One sentence (of length less than 20) per line:<br>
         ```beside her ,
         i
         people might not know```<br>
    The `<bos>` symbol is not explicitly in the file, but you should still use it as the first input.<br>
    __Required output format groupXX.continuation__ (where XX is your __group number__)<br>
         ```beside her , something happened ! <eos>
         i do n’t recall making a noise , but i must have , because bob just looked up from his
         people might not know the answer . <eos>```
    - You have to submit at https://cmt3.research.microsoft.com/NLUETHZ2018


__Infrastructure__

You must use Tensorflow, but any programming language is allowed. However, we strongly recommend `python3`. You have access to two compute resources: Unlimited CPU usage on Euler and GPU usage on Leonhard. Note that the difference in speed is typically a factor between 10 and 100.

## Code for Task 1
### Task 1.1

In [6]:
import tensorflow as tf
import numpy as np

from helpers.load_embeddings import load_embedding

%matplotlib inline

In [15]:
# Read training and evalutation data from files

with open('./data/sentences.train') as f:
    train_set = f.readlines()
    
with open('./data/sentences.eval') as f:
    eval_set = f.readlines()

In [16]:
# Have a random peek at the given data

print('Training set:\n')
for i in np.random.randint(0, len(train_set), 10):
    print(train_set[i])

print('\n Evaluation set:\n')
for i in np.random.randint(0, len(eval_set), 10):
    print(eval_set[i])

Training set:

i nodded and walked toward the door , confused .

eamon was slightly relieved .

toby went into the other room , gathered up the two guns , and went out the back door of his building .

i 'm waiting to hear my punishment , cassie thought .

`` feathers ? ''

she let out an amused `` heh '' at him .

`` i nodded all to vigorously .

i keep craving pancakes . ''

me .

you , too , abigail .


 Evaluation set:

`` sure , there are stories , '' gray said .

something that made her personal life very complicated .

she had the last three of them at home with a certified mid-wife , and home-schooled them all way ahead of her time in her small texas town .

owen growled .

normal people did n't kill for a living and normal people did n't sleep with the most notorious assassin to ever live and not turn him in .

the massive roots encircled the area like walls , while tightly woven branches made a ceiling above them .

he sighed as he flopped back onto the bed .

oh no , she bega